### Parsing a Text File to Find Wikipedia Links

The case when we have also Birth and Death dates to help improve disambiguation

In [ ]:
import re
from collections import Counter

def pre_process_list(filepath: str):
    with open(filepath) as f:
        rows = f.readlines()

    life_pattern = r"(\d+)\s+(–|-)\s+(\d+)"
    people = []
    for row in rows:
        if '(' in row and ')' in row:
            bracket_open_index = row.index('(')
            bracket_closed_index = row.index(')')
            name = row[:bracket_open_index].strip()
            lifespan = row[bracket_open_index:bracket_closed_index+1].strip()
            if len(row) > bracket_closed_index+1:
                description = row[bracket_closed_index+1:].strip()
            else:
                description = ""
            m = re.search(life_pattern, lifespan)
            if m:
                birth = m.group(1).strip()
                death = m.group(3).strip()
            else:
                birth = -1
                death = -1
            people.append((name, int(birth), int(death)))
            # print("Name:", name)
            # print("Lifespan:", lifespan, birth, death)
            # print("Description:", description)
            # print("---")
        else:
            print("MISSED", row)
    
    return people

def pre_process_latinamerica_list(filepath: str):
    with open(filepath) as f:
        rows = f.readlines()
    people = []
    countries = []
    for row in rows:
        columns = row.split('|')
        name = columns[0].strip()
        country = columns[1].strip()
        countries.append(country)
        birth = -1
        death = -1
        people.append((name, int(birth), int(death)))
    
    [print(x) for x in Counter(countries).most_common()]
    return people


In [ ]:
from utils.utils_wiki import get_wikipedia_article, save_wikipedia_page
import time, os
from typing import List, Tuple
from urllib.parse import unquote

def create_ready_filelist(people: List[Tuple], output_list: str, output_dir: str):
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    with open(output_list, "w") as f:
        for name, birth, death in people:
            page = get_wikipedia_article(name, query_restrictions={'birth_year': birth, 'death_year': death})
            if page:
                save_wikipedia_page(page, output_path=f"{output_dir}/{page.title.replace(' ', '_').lower()}.txt", include_metadata=True, include_sections=True, include_infobox=True)
                f.write(f"{page.title} | {page.url}\n")
            time.sleep(5)

def get_person_name(wiki_link: str):
    if "/" not in wiki_link: return None
    name_url = wiki_link.split("/")[-1]
    person_name = unquote(name_url)
    return person_name.replace("_", " ")

def get_pages_from_ready(filepath: str, output_dir: str):
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    wiki_titles = []
    with open(filepath) as f:
        for line in f:
            elems = line.split("|")
            person_wiki_name = get_person_name(elems[1].strip())
            wiki_titles.append(person_wiki_name)
    for title in wiki_titles:
        page = get_wikipedia_article(title)
        if page:
            save_wikipedia_page(page, output_path=f"{output_dir}/{page.title.replace(' ', '_').lower()}.txt", include_metadata=True, include_sections=True, include_infobox=True)
        time.sleep(5)

# women_famous = pre_process_list("resources/15_famous_women.txt")
# create_ready_filelist(women_famous, "resources/15_famous_women.ready.txt", "data/wikipedia/top_women")

# get_pages_from_ready("resources/12_activists_lgbtq.ready.txt", "data/wikipedia/top_lgbtq")

# get_pages_from_ready("resources/50_famous_mexico.ready.txt", "data/wikipedia/top_mexico")

# top_100_people = pre_process_list("resources/top_100_world_most_influential.txt")
# create_ready_filelist(top_100_people, "resources/top_100_world_most_influential.ready.txt", "data/wikipedia/top100")

# get_pages_from_ready("resources/100_famous_latinamerica.ready.txt", "data/wikipedia/top_latinamerica")

### Parsing a Wikipedia Page with a Table

The case of the "List of women explorers and travelers" [Link](https://en.wikipedia.org/wiki/List_of_women_explorers_and_travelers)

In [15]:
from utils.utils_wiki import get_wikipedia_article, save_wikipedia_page
import pandas as pd
import time, os

def get_pages_from_dataframe(df: pd.DataFrame, output_dir: str):
    if not os.path.exists(output_dir): os.makedirs(output_dir)

    for ix, row in table.iterrows():
        # Get Name
        person_name = row["Name"]
        # Get valid birth date
        try:
            birth = int(row["Born"])
        except:
            birth = -1
        # Get valid Death date
        try:
            death = int(row["Died"])
        except:
            death = -1
        page = get_wikipedia_article(person_name, query_restrictions={'birth_year': birth, 'death_year': death})
        if page:
            save_wikipedia_page(page, output_path=f"{output_dir}/{page.title.replace(' ', '_').lower()}.txt", include_metadata=True, include_sections=True, include_infobox=True)
        time.sleep(5)


table = pd.read_html("https://en.wikipedia.org/wiki/List_of_women_explorers_and_travelers")[0]
print(table.head())

# output_dir = "data/wikipedia/women_explorers"
# get_pages_from_dataframe(table, output_dir)



                              Name Nationality  Born  Died  \
0           Harriet Chalmers Adams    American  1875  1937   
1                       Tania Aebi    American  1966   NaN   
2                    Alexis Alford    American  1998   NaN   
3                   Felicity Aston     British  1977   NaN   
4  Florence Augusta Merriam Bailey    American  1863  1948   

                                       Achievement/s  
0  Explored and photographed South America, Asia,...  
1  Held the record as the youngest person and fir...  
2  Youngest person to travel to every country in ...  
3  First person to ski alone across the Antarctic...  
4  American ornithologist and nature writer, who ...  
